In [1]:
from google.colab import userdata
import os

api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

os.environ["OPENAI_API_BASE"] = "https://api.aimlapi.com/v1"

In [2]:
!pip install -q langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.7 MB/s eta 0:00:00


In [3]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnablePassthrough

##Define Each Component

In [4]:
# Prompt template – accepts a variable {topic}
prompt = ChatPromptTemplate.from_template("Write a short haiku about {topic}.")

# Model
model = ChatOpenAI(model="gpt-3.5-turbo")

# Parser
parser = StrOutputParser()

##Combine Using RunnableSequence
###Explicitly defining flow

In [5]:
# Build a sequence manually
chain = RunnableSequence(first=prompt, middle=[model], last=parser)

# Run it
result = chain.invoke({"topic": "autumn leaves"})
print(result)

Crisp leaves turn to gold
Dancing in the autumn breeze
Nature's painted scene


##Inspect Intermediate Steps
###You can insert a RunnablePassthrough to peek at data as it flows.

In [6]:
# RunnablePassthrough lets data “pass through” unchanged
debug_chain = RunnableSequence(
    first=prompt,
    middle=[RunnablePassthrough(), model],
    last=parser
)

# Try running it
result = debug_chain.invoke({"topic": "mountains"})
print(result)

Majestic peaks soar high
Silent guardians of earth
Beauty beyond words


##Use itemgetter to Extract or Route Data

###When your inputs/outputs are dicts, itemgetter lets you pick pieces.

In [7]:
from pprint import pprint

class DebugPassthrough(RunnablePassthrough):
    def invoke(self, input, config=None):
        pprint({"DEBUG INPUT": input})
        return super().invoke(input, config)

debug_chain = RunnableSequence(
    first=prompt,
    middle=[DebugPassthrough(), model],
    last=parser
)

result = debug_chain.invoke({"topic": "rain"})
print(result)

{'DEBUG INPUT': ChatPromptValue(messages=[HumanMessage(content='Write a short haiku about rain.', additional_kwargs={}, response_metadata={})])}
Raindrops fall softly
Quenching the earth's thirst with love
Life blooms anew here


In [8]:
# A prompt needing both topic and mood
prompt = ChatPromptTemplate.from_template("Write a {mood} poem about {topic}.")

In [9]:
chain = (
    {
        "topic": itemgetter("topic"),
        "mood": itemgetter("mood")
    }
    | prompt
    | model
    | parser
)

result = chain.invoke({"topic": "stars", "mood": "melancholic"})
print(result)

Oh, how the stars do weep,
Their shimmering light fading,
As the darkness slowly creeps,
And the night grows ever shading.

Each twinkle once so bright,
Now dulled with sorrow's haze,
As they fade into the night,
Lost in the endless maze.

The heavens weep for lost dreams,
For wishes left unspoken,
An endless sea of tears it seems,
Their beauty forever broken.

Each star a reminder of the past,
A flicker of hope extinguished,
As they watch the world go by so fast,
Their brilliance forever diminished.

Oh, how the stars do weep,
Their light now a distant memory,
Lost in the shadows deep,
Forever lost in eternity.
